<a href="https://colab.research.google.com/github/shiweijing/Coursera_Capstone/blob/master/Toronto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neighborhoods in Toronto**

In [1]:
import numpy as np
import pandas as pd

In [2]:
#import Neighbour of Toronto from WIKIPEDIA website
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
print(tables[0])

    Postal Code  ...                                      Neighbourhood
0           M1A  ...                                       Not assigned
1           M2A  ...                                       Not assigned
2           M3A  ...                                          Parkwoods
3           M4A  ...                                   Victoria Village
4           M5A  ...                          Regent Park, Harbourfront
..          ...  ...                                                ...
175         M5Z  ...                                       Not assigned
176         M6Z  ...                                       Not assigned
177         M7Z  ...                                       Not assigned
178         M8Z  ...  Mimico NW, The Queensway West, South of Bloor,...
179         M9Z  ...                                       Not assigned

[180 rows x 3 columns]


In [8]:
#remove rows with borough = Not assigned
df=tables[0]
df.head()
df=df[df['Borough']!='Not assigned']
print(df)

    Postal Code  ...                                      Neighbourhood
2           M3A  ...                                          Parkwoods
3           M4A  ...                                   Victoria Village
4           M5A  ...                          Regent Park, Harbourfront
5           M6A  ...                   Lawrence Manor, Lawrence Heights
6           M7A  ...        Queen's Park, Ontario Provincial Government
..          ...  ...                                                ...
160         M8X  ...      The Kingsway, Montgomery Road, Old Mill North
165         M4Y  ...                               Church and Wellesley
168         M7Y  ...  Business reply mail Processing Centre, South C...
169         M8Y  ...  Old Mill South, King's Mill Park, Sunnylea, Hu...
178         M8Z  ...  Mimico NW, The Queensway West, South of Bloor,...

[103 rows x 3 columns]


In [14]:
#check if there is cell where neighbourhood is not assigned
df[df['Neighbourhood']=='Not assigned']

,Postal Code,Borough,Neighbourhood


print the shape of the dataframe

In [15]:
df.shape

(103, 3)